##Next Word Prediction

In [1]:
with open("/content/normalized.txt", "r") as file:
  data = file.read()
file.close()

In [2]:
data

'there were little things that she simply could not stand\nthe sound of someone tapping their nails on the table\na person chewing with their mouth open\nanother human imposing themselves into her space\nshe couldnt stand any of these things but none of them compared to the number one thing she couldnt stand which topped all of them combined\nit went through such rapid contortions that the little bear was forced to change his hold on it so many times he became confused in the darkness and could not for the life of him tell whether he held the sheep right side up or upside down\nbut that point was decided for him a moment later by the animal itself who with a sudden twist jabbed its horns so hard into his lowest ribs that he gave a grunt of anger and disgust\nthe cab arrived late\nthe inside was in as bad of shape as the outside which was concerning and it didnt appear that it had been cleaned in months\nthe green tree airfreshener hanging from the rearview mirror was either exhausted o

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
tokenizer = Tokenizer()

data = data.lower().split("\n")

In [5]:
len(data)

10349

In [6]:
tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1

In [7]:
print(total_words)

2011


In [8]:
import io
import json

tokenizer_json = tokenizer.to_json()
with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [10]:
input_sequences = []
for line in data:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i + 1]
    input_sequences.append(n_gram_sequence)

In [11]:
print("Sequences : ",len(input_sequences))

Sequences :  139168


In [12]:
max_sequence_len = max([len(x) for x in input_sequences])

In [13]:
print("Maximum Length : ", max_sequence_len)

Maximum Length :  52


In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [15]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


In [16]:
xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]

In [17]:
print(len(xs), len(labels))

139168 139168


In [18]:
import tensorflow as tf

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [19]:
ys[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.optimizers import Adam

In [26]:
model = Sequential()
model.add(Embedding(total_words, 240, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Bidirectional(LSTM(75)))
model.add(Dense(total_words, activation="softmax"))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 51, 240)           482640    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 51, 300)           469200    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 150)               225600    
_________________________________________________________________
dense_2 (Dense)              (None, 2011)              303661    
Total params: 1,481,101
Trainable params: 1,481,101
Non-trainable params: 0
_________________________________________________________________


In [28]:
history = model.fit(xs, ys, epochs=20, verbose=1)

Epoch 1/20
4349/4349 [==============================] - 84s 18ms/step - loss: 5.6206 - accuracy: 0.0993
Epoch 2/20
4349/4349 [==============================] - 78s 18ms/step - loss: 3.1726 - accuracy: 0.3005
Epoch 3/20
4349/4349 [==============================] - 78s 18ms/step - loss: 2.1751 - accuracy: 0.4634
Epoch 4/20
4349/4349 [==============================] - 78s 18ms/step - loss: 1.8791 - accuracy: 0.5239
Epoch 5/20
4349/4349 [==============================] - 79s 18ms/step - loss: 1.7159 - accuracy: 0.5601
Epoch 6/20
4349/4349 [==============================] - 79s 18ms/step - loss: 1.6065 - accuracy: 0.5854
Epoch 7/20
4349/4349 [==============================] - 79s 18ms/step - loss: 1.5631 - accuracy: 0.5955
Epoch 8/20
4349/4349 [==============================] - 78s 18ms/step - loss: 1.5439 - accuracy: 0.5999
Epoch 9/20
4349/4349 [==============================] - 78s 18ms/step - loss: 1.5619 - accuracy: 0.5994
Epoch 10/20
4349/4349 [==============================] - 79s 18m

In [29]:
seed_text = "cover"
next_words = 20
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


cover his breath let this the ground as the consequences as he tried to the same thing he smiled than showing


In [30]:
model.save("next_word.h5")

In [31]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)